In [1]:
#Calling a method of a class in Python (a = MyClass(x), x.func(y)) 
#corresponds to a function call in Julia, e.g. a = MyStruct(x), func(x::MyStruct, y). 
#In general, multiple dispatch is more flexible and powerful than the Python class system.
using Distributions
struct Bandits
    p::Vector{Float64}
    optimal
    _len_
    Bandits(p) = new(p,maximum(p),length(p));
end 
pull(p_array,i) = rand() < p_array[i]

Base.@kwdef mutable struct BayesianStrategy
    bandits::Bandits
    n_bandits
    wins
    trials
    N
    choices
    bb_score
    BayesianStrategy(bandits) = new(bandits,bandits._len_,zeros(bandits._len_),zeros(bandits._len_),0,[],[]);
end

function sample_bandits(B::BayesianStrategy,n = 1)
    modified_B = B
    bb_score = zeros(n)
    choices  = zeros(n)
    for k in 1:n
        #sample from the bandits's priors, and select the largest sample
        choice = argmax(rand.(Beta.(1 .+ modified_B.wins ,1 .+ modified_B.trials - modified_B.wins)))
        
        #sample the chosen bandit
        result = pull(modified_B.bandits.p,choice)
        
        #update priors and score
        modified_B.wins[choice]   += result
        modified_B.trials[choice] += 1
        bb_score[k]      = result
        modified_B.N              += 1
        choices[k]       = choice
    end
    append!( modified_B.bb_score, bb_score )
    append!( modified_B.choices, choices )
    return modified_B
end

sample_bandits (generic function with 2 methods)

In [2]:
methods(Vector)

# 5 methods for type constructor:
[1] (Vector{T} where T)() in Base at baseext.jl:27
[2] (Vector{T} where T)(D::SuiteSparse.CHOLMOD.Dense{T}) where T in SuiteSparse.CHOLMOD at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\SuiteSparse\src\cholmod.jl:997
[3] (Vector{T} where T)(x::SparseArrays.AbstractSparseVector{Tv, Ti} where Ti) where Tv in SparseArrays at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\SparseArrays\src\sparsevector.jl:936
[4] (Array{T, N} where T)(x::AbstractArray{S, N}) where {S, N} in Core at boot.jl:470
[5] (Vector{T} where T)(::UndefInitializer, m::Integer) in Base at baseext.jl:21

In [2]:
x = [0.1,0.2,0.3,0.4,0.5]
p = Bandits(x)

Bandits([0.1, 0.2, 0.3, 0.4, 0.5], 0.5, 5)

In [22]:
z = BayesianStrategy(p)
sample_bandits(z,1)

BayesianStrategy(Bandits([0.1, 0.2, 0.3, 0.4, 0.5], 0.5, 5), 5, [0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 1.0], 1, Any[5.0], Any[1.0])

In [2]:
x = [1,2,3,4,5]
p = Bandits(x)
z = BayesianStrategy(p)

BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])

In [3]:
y = Array{BayesianStrategy,1}(undef, 5)

5-element Vector{BayesianStrategy}:
 #undef
 #undef
 #undef
 #undef
 #undef

In [3]:
V = Vector{BayesianStrategy}(undef, 5)

5-element Vector{BayesianStrategy}:
 #undef
 #undef
 #undef
 #undef
 #undef

In [5]:
M = Matrix{BayesianStrategy}(undef, 2, 2)

2×2 Matrix{BayesianStrategy}:
 #undef  #undef
 #undef  #undef

In [6]:
zeros(1)

1-element Vector{Float64}:
 0.0

In [7]:
 zeros(Int8, 2, 3)

2×3 Matrix{Int8}:
 0  0  0
 0  0  0

In [8]:
fill(zeros(1), (2,2))

2×2 Matrix{Vector{Float64}}:
 [0.0]  [0.0]
 [0.0]  [0.0]

In [9]:
methods(fill!)

# 14 methods for generic function "fill!":
[1] fill!(A::Union{LinearAlgebra.Hermitian{T, S}, LinearAlgebra.Symmetric{T, S}} where {T, S}, x) in LinearAlgebra at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\LinearAlgebra\src\symmetric.jl:303
[2] fill!(A::Union{LinearAlgebra.Bidiagonal, LinearAlgebra.Diagonal, LinearAlgebra.SymTridiagonal, LinearAlgebra.Tridiagonal}, x) in LinearAlgebra at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\LinearAlgebra\src\special.jl:298
[3] fill!(A::Union{SparseArrays.AbstractSparseMatrixCSC, SparseArrays.SparseVector}, x) in SparseArrays at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\SparseArrays\src\sparsevector.jl:2062
[4] fill!(cb::DataStructures.CircularBuffer, data) in DataStructures at C:\Users\91863\.julia\packages\DataStructures\ixwFs\src\circular_buffer.jl:143
[5] fill!(S::SharedArrays.SharedArray, v) in SharedArrays at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\SharedArrays\src\SharedArrays.jl:514
[6] fill!(B::BitArray, x) in Base at bitarray.jl:377
[7] fill!(a::Union{Array{Int8, N} where N, Array{UInt8, N} where N}, x::Integer) in Base at array.jl:405
[8] fill!(dest::Array{T, N} where N, x) where T in Base at array.jl:332
[9] fill!(B::Base.Broadcast.BitMaskedBitArray, b::Bool) in Base.Broadcast at broadcast.jl:1187
[10] fill!(F::FillArrays.AbstractFill, v) in FillArrays at C:\Users\91863\.julia\packages\FillArrays\VLeUk\src\FillArrays.jl:55
[11] fill!(V::SubArray{Bool, var"#s832", var"#s831", Tuple{AbstractUnitRange{Int64}}, L} where {var"#s832", var"#s831"<:BitArray, L}, x) in Base at multidimensional.jl:1421
[12] fill!(V::SubArray{Bool, var"#s832", var"#s831", Tuple{AbstractUnitRange{Int64}, Vararg{Union{Int64, AbstractUnitRange{Int64}}, N} where N}, L} where {var"#s832", var"#s831"<:BitArray, L}, x) in Base at multidimensional.jl:1430
[13] fill!(V::SubArray{Tv, var"#s832", var"#s831", var"#s830", L} where {var"#s832", var"#s831"<:(SparseArrays.AbstractSparseMatrixCSC{Tv, Ti} where Ti<:Integer), var"#s830"<:Tuple{Union{Integer, AbstractVector{var"#s829"} where var"#s829"<:Integer}, Union{Integer, AbstractVector{var"#s829"} where var"#s829"<:Integer}}, L}, x) where Tv in SparseArrays at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\SparseArrays\src\sparsematrix.jl:2620
[14] fill!(A::AbstractArray{T, N} where N, x) where T in Base at multidimensional.jl:1054

In [10]:
fill!(V,z)

5-element Vector{BayesianStrategy}:
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])

In [11]:
fill!(M,z)

2×2 Matrix{BayesianStrategy}:
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])  …  BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])     BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])

In [29]:
methods(zero)

# 31 methods for generic function "zero":
[1] zero(::Union{Type{P}, P}) where P<:Dates.Period in Dates at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\Dates\src\periods.jl:53
[2] zero(a::SparseArrays.AbstractSparseArray) in SparseArrays at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\SparseArrays\src\SparseArrays.jl:55
[3] zero(::AbstractIrrational) in Base at irrationals.jl:148
[4] zero(r::FillArrays.Zeros{T, N, Axes} where Axes) where {T, N} in FillArrays at C:\Users\91863\.julia\packages\FillArrays\VLeUk\src\FillArrays.jl:539
[5] zero(J::LinearAlgebra.UniformScaling{T}) where T in LinearAlgebra at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\LinearAlgebra\src\uniformscaling.jl:136
[6] zero(h::StatsBase.Histogram{T, N, E}) where {T, N, E} in StatsBase at C:\Users\91863\.julia\packages\StatsBase\DU1bT\src\hist.jl:538
[7] zero(r::FillArrays.Fill{T, N, Axes} where Axes) where {T, N} in FillArrays at C:\Users\91863\.julia\packages\FillArrays\VLeUk\src\FillArrays.jl:541
[8] zero(::T) where T<:Dates.TimeType in Dates at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\Dates\src\types.jl:423
[9] zero(x::Number) in Base at number.jl:260
[10] zero(::CartesianIndex{N}) where N in Base.IteratorsMD at multidimensional.jl:106
[11] zero(r::FillArrays.Ones{T, N, Axes} where Axes) where {T, N} in FillArrays at C:\Users\91863\.julia\packages\FillArrays\VLeUk\src\FillArrays.jl:540
[12] zero(::Type{var"#s832"} where var"#s832"<:AbstractIrrational) in Base at irrationals.jl:149
[13] zero(::Type{Base.TwicePrecision{T}}) where T in Base at twiceprecision.jl:260
[14] zero(::Type{LinearAlgebra.UniformScaling{T}}) where T in LinearAlgebra at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\LinearAlgebra\src\uniformscaling.jl:135
[15] zero(::Type{var"#s15"} where var"#s15"<:ChainRulesCore.NotImplemented) in ChainRulesCore at C:\Users\91863\.julia\packages\ChainRulesCore\BYuIz\src\differentials\notimplemented.jl:47
[16] zero(::Type{T}) where T<:Number in Base at number.jl:261
[17] zero(::Type{CartesianIndex{N}}) where N in Base.IteratorsMD at multidimensional.jl:107
[18] zero(::Type{var"#s4"} where var"#s4"<:ChainRulesCore.AbstractTangent) in ChainRulesCore at C:\Users\91863\.julia\packages\ChainRulesCore\BYuIz\src\differentials\abstract_zero.jl:43
[19] zero(x::ChainRulesCore.NotImplemented) in ChainRulesCore at C:\Users\91863\.julia\packages\ChainRulesCore\BYuIz\src\differentials\notimplemented.jl:46
[20] zero(::ChainRulesCore.AbstractTangent) in ChainRulesCore at C:\Users\91863\.julia\packages\ChainRulesCore\BYuIz\src\differentials\abstract_zero.jl:42
[21] zero(::Missing) in Base at missing.jl:101
[22] zero(x::DataStructures.DiBitVector) in DataStructures at C:\Users\91863\.julia\packages\DataStructures\ixwFs\src\dibit_vector.jl:80
[23] zero(x::AbstractArray{T, N} where N) where T in Base at abstractarray.jl:1085
[24] zero(::Type{Dates.DateTime}) in Dates at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\Dates\src\types.jl:420
[25] zero(::Type{Missing}) in Base at missing.jl:104
[26] zero(::Type{Dates.Date}) in Dates at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\Dates\src\types.jl:421
[27] zero(::Type{Pkg.Resolve.FieldValue}) in Pkg.Resolve at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\Pkg\src\Resolve\fieldvalues.jl:38
[28] zero(::Type{Pkg.Resolve.VersionWeight}) in Pkg.Resolve at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\Pkg\src\Resolve\versionweights.jl:15
[29] zero(::Type{LibGit2.GitHash}) in LibGit2 at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\LibGit2\src\oid.jl:221
[30] zero(::Type{Dates.Time}) in Dates at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\Dates\src\types.jl:422
[31] zero(::Type{Union{Missing, T}}) where T in Base at mis

In [31]:
zero(2.0)

0.0

In [12]:
Base.zero(::Type{Bandits},N) = Bandits(zeros(N))

In [13]:
a = zero(Bandits,8)

Bandits([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.0, 8)

In [14]:
M = Matrix{Bandits}(undef,2,2)

2×2 Matrix{Bandits}:
 #undef  #undef
 #undef  #undef

In [15]:
fill!(M,a)

2×2 Matrix{Bandits}:
 Bandits([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.0, 8)  …  Bandits([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.0, 8)
 Bandits([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.0, 8)     Bandits([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.0, 8)

In [16]:
typeof(fill!(M,a))

Matrix{Bandits} (alias for Array{Bandits, 2})

Use vectorisation

In [2]:
x = [1,2,3,4,5]
p = Bandits(x)
z = BayesianStrategy(p)
z1 = BayesianStrategy(p)

BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])

In [36]:
V = Vector{BayesianStrategy}(undef, 5)

5-element Vector{BayesianStrategy}:
 #undef
 #undef
 #undef
 #undef
 #undef

In [4]:
A = fill!(V,z)

5-element Vector{BayesianStrategy}:
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])

In [3]:
sample_bandits(z1,1)

BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [1.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0], 1, Any[1.0], Any[1.0])

In [56]:
broadcast(sample_bandits, A, 1)

5-element Vector{BayesianStrategy}:
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 2.0, 1.0, 0.0, 2.0], [0.0, 2.0, 1.0, 0.0, 2.0], 5, Any[5.0, 3.0, 2.0, 5.0, 2.0], Any[1.0, 1.0, 1.0, 1.0, 1.0])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 2.0, 1.0, 0.0, 2.0], [0.0, 2.0, 1.0, 0.0, 2.0], 5, Any[5.0, 3.0, 2.0, 5.0, 2.0], Any[1.0, 1.0, 1.0, 1.0, 1.0])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 2.0, 1.0, 0.0, 2.0], [0.0, 2.0, 1.0, 0.0, 2.0], 5, Any[5.0, 3.0, 2.0, 5.0, 2.0], Any[1.0, 1.0, 1.0, 1.0, 1.0])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 2.0, 1.0, 0.0, 2.0], [0.0, 2.0, 1.0, 0.0, 2.0], 5, Any[5.0, 3.0, 2.0, 5.0, 2.0], Any[1.0, 1.0, 1.0, 1.0, 1.0])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 2.0, 1.0, 0.0, 2.0], [0.0, 2.0, 1.0, 0.0, 2.0], 5, Any[5.0, 3.0, 2.0, 5.0, 2.0], Any[1.0, 1.0, 1.0, 1.0, 1.0])

The above answer is wrong

In [5]:
sample_bandits.(A,1)

5-element Vector{BayesianStrategy}:
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 1.0, 4.0, 0.0, 0.0], [0.0, 1.0, 4.0, 0.0, 0.0], 5, Any[2.0, 3.0, 3.0, 3.0, 3.0], Any[1.0, 1.0, 1.0, 1.0, 1.0])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 1.0, 4.0, 0.0, 0.0], [0.0, 1.0, 4.0, 0.0, 0.0], 5, Any[2.0, 3.0, 3.0, 3.0, 3.0], Any[1.0, 1.0, 1.0, 1.0, 1.0])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 1.0, 4.0, 0.0, 0.0], [0.0, 1.0, 4.0, 0.0, 0.0], 5, Any[2.0, 3.0, 3.0, 3.0, 3.0], Any[1.0, 1.0, 1.0, 1.0, 1.0])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 1.0, 4.0, 0.0, 0.0], [0.0, 1.0, 4.0, 0.0, 0.0], 5, Any[2.0, 3.0, 3.0, 3.0, 3.0], Any[1.0, 1.0, 1.0, 1.0, 1.0])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 1.0, 4.0, 0.0, 0.0], [0.0, 1.0, 4.0, 0.0, 0.0], 5, Any[2.0, 3.0, 3.0, 3.0, 3.0], Any[1.0, 1.0, 1.0, 1.0, 1.0])

The above answer is wrong

Why wat happened ? wat is the solution

it all has to do with pointers ! when you are passing a object in Julia u pass it by reference

this means once you create an object and start to copy them :-
you dont create new objects u just create multiple arrows referring to the same object

In [28]:
x = [1,2,3,4,5]
p = Bandits(x)

Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5)

In [12]:
V = []

Any[]

In [20]:
methods(Vector)

# 5 methods for type constructor:
[1] (Vector{T} where T)() in Base at baseext.jl:27
[2] (Vector{T} where T)(D::SuiteSparse.CHOLMOD.Dense{T}) where T in SuiteSparse.CHOLMOD at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\SuiteSparse\src\cholmod.jl:997
[3] (Vector{T} where T)(x::SparseArrays.AbstractSparseVector{Tv, Ti} where Ti) where Tv in SparseArrays at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\SparseArrays\src\sparsevector.jl:936
[4] (Array{T, N} where T)(x::AbstractArray{S, N}) where {S, N} in Core at boot.jl:470
[5] (Vector{T} where T)(::UndefInitializer, m::Integer) in Base at baseext.jl:21

In [14]:
V = [x for i in 1:5]

5-element Vector{Vector{Int64}}:
 [1, 2, 3, 4, 5]
 [1, 2, 3, 4, 5]
 [1, 2, 3, 4, 5]
 [1, 2, 3, 4, 5]
 [1, 2, 3, 4, 5]

lets try to take this apart one bit at a time

Let's take a look at the methods of Vector

(Vector{T} where T)() in Base at baseext.jl:27 
This implies we can pass any kind of Type to it without declaring the Type

In [19]:
Y = Bandits.(V)

5-element Vector{Bandits}:
 Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5)
 Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5)
 Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5)
 Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5)
 Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5)

Viola ! Julia understood that it needs to create a vector of Bandits without having to encounter this type before

Lets push it further!!!

In [26]:
C = BayesianStrategy.(Y)

5-element Vector{BayesianStrategy}:
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])

Now we can see a Vector of BayesianStrategy !

We know we can use broadcasting (taking advantage of vectorisation) by simply using '.' operator .

sample_bandits takes input of BayesianStrategy type(not a vector !)
Lets try to see if it works ?

In [27]:
sample_bandits.(C,1)

5-element Vector{BayesianStrategy}:
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0], 1, Any[4.0], Any[1.0])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0], 1, Any[4.0], Any[1.0])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 1.0], 1, Any[5.0], Any[1.0])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0], 1, Any[4.0], Any[1.0])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0], 1, Any[3.0], Any[1.0])

In [30]:
X = BayesianStrategy.(Bandits.(V))

5-element Vector{BayesianStrategy}:
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])

In [18]:
sample_bandits.(X,1)

5-element Vector{BayesianStrategy}:
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0], 1, Any[3.0], Any[1.0])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0], 1, Any[2.0], Any[1.0])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [1.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0, 0.0], 1, Any[1.0], Any[1.0])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0], 1, Any[4.0], Any[1.0])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0], 1, Any[4.0], Any[1.0])

Lets complicate this problem further ! coz y not :p

In [28]:
using Distributions
function sample_bandits_other_strats(B::BayesianStrategy,n = 1,strats = "bayesian_bandit_choice")
    bb_score = zeros(n)
    choices  = zeros(n)
    for k in 1:n
        #sample from the bandits's priors, and select the largest sample
        #choice_function chooses the strats depending on our choice
        choice = choice_function(B,strats)
        
        #sample the chosen bandit
        result = pull(B.bandits.p,choice)
        
        #update priors and score
        B.wins[choice]   += result
        B.trials[choice] += 1
        bb_score[k]      = result
        B.N              += 1
        choices[k]       = choice
    end
    append!( B.bb_score, bb_score )
    append!( B.choices, choices )
    return B
end

function choice_function(B::BayesianStrategy,strats)
    if strats == "bayesian_bandit_choice"
        #bayesian_bandit_choice
        choice = argmax(rand.(Beta.(1 .+ B.wins ,1 .+ B.trials - B.wins)))
        
    elseif strats == "max_mean"
        #max_mean : pick the bandit with the current best observed proportion of winning 
        choice = argmax(B.wins ./ (B.trials .+ 1));
        
    elseif strats == "lower_credible_choice"
        #pick the bandit with the best LOWER BOUND
        a = B.wins .+ 1
        b = 1 .+ B.trials - B.wins
        lb = (a./(a+b)) .- 1.65.*sqrt.((a.*b)./( ((a+b).^2).*(a.+b.+1) ))
        choice = argmax(lb)
        
    elseif strats == "upper_credible_choice"
        a = 1 .+ B.wins
        b = 1 .+ B.trials - B.wins
        lb = (a./(a.+b)) .+ 1.65.*sqrt.((a.*b)./( ((a+b).^2).*(a.+b.+1) ))
        choice = argmax(lb)
        
    elseif strats == "ucb_bayes"
        a = B.wins .+ 1
        b = 1 .+ B.trials - B.wins
        alpha = 1- 1.0/(B.N+1)
        choice = argmax(quantile.(Beta.(a,b),alpha))
        
    elseif strats == "random_choice"
        choice = rand(1:length(B.wins));
    else
        choice = []
    end
    return choice
end

regret(probabilities, choices) = cumsum(maximum(probabilities) .- probabilities[convert( Vector{Int},choices)])

regret (generic function with 1 method)

In [31]:
X = BayesianStrategy.(Bandits.(V))
X

5-element Vector{BayesianStrategy}:
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0], 0, Any[], Any[])

In [32]:
strategies= ["upper_credible_choice", 
            "bayesian_bandit_choice", 
            "ucb_bayes" , 
            "max_mean",
            "random_choice"]

5-element Vector{String}:
 "upper_credible_choice"
 "bayesian_bandit_choice"
 "ucb_bayes"
 "max_mean"
 "random_choice"

In [33]:
 sample_bandits_other_strats.(X,100,strategies)

5-element Vector{BayesianStrategy}:
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [100.0, 0.0, 0.0, 0.0, 0.0], [100.0, 0.0, 0.0, 0.0, 0.0], 100, Any[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], Any[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [0.0, 21.0, 14.0, 4.0, 61.0], [0.0, 21.0, 14.0, 4.0, 61.0], 100, Any[5.0, 5.0, 5.0, 3.0, 5.0, 2.0, 5.0, 5.0, 5.0, 2.0  …  5.0, 3.0, 2.0, 5.0, 2.0, 5.0, 5.0, 2.0, 3.0, 3.0], Any[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
 BayesianStrategy(Bandits([1.0, 2.0, 3.0, 4.0, 5.0], 5, 5), 5, [100.0, 0.0, 0.0, 0.0, 0.0], [100.0, 0.0, 0.0, 0.0, 0.0], 100, Any[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], Any[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 

In [4]:
wins = [17.0, 17.0, 23.0, 17.0, 26.0]
trials =  [17.0, 17.0, 23.0, 17.0, 26.0]

5-element Vector{Float64}:
 17.0
 17.0
 23.0
 17.0
 26.0

In [6]:
Beta(1 + wins[1], 1 + trials[1] - wins[1])

Beta{Float64}(α=18.0, β=1.0)

In [5]:
using Distributions

In [8]:
1 .+ wins

5-element Vector{Float64}:
 18.0
 18.0
 24.0
 18.0
 27.0

In [9]:
1 .+ trials - wins

5-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0

In [10]:
Beta.(1 .+ wins, 1 .+ trials - wins)

5-element Vector{Beta{Float64}}:
 Beta{Float64}(α=18.0, β=1.0)
 Beta{Float64}(α=18.0, β=1.0)
 Beta{Float64}(α=24.0, β=1.0)
 Beta{Float64}(α=18.0, β=1.0)
 Beta{Float64}(α=27.0, β=1.0)

In [11]:
rand.(Beta.(1 .+ wins, 1 .+ trials - wins))

5-element Vector{Float64}:
 0.9502803101535015
 0.9420562776741263
 0.9062253739570402
 0.9987616143052866
 0.9946406245943975

In [21]:
argmax(rand.(Beta.(1 .+ wins, 1 .+ trials - wins)))

4

In [23]:
methods(+)

# 271 methods for generic function "+":
[1] +(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:87
[2] +(c::Union{UInt16, UInt32, UInt8}, x::BigInt) in Base.GMP at gmp.jl:528
[3] +(c::Union{Int16, Int32, Int8}, x::BigInt) in Base.GMP at gmp.jl:534
[4] +(c::Union{UInt16, UInt32, UInt8}, x::BigFloat) in Base.MPFR at mpfr.jl:376
[5] +(c::Union{Int16, Int32, Int8}, x::BigFloat) in Base.MPFR at mpfr.jl:384
[6] +(c::Union{Float16, Float32, Float64}, x::BigFloat) in Base.MPFR at mpfr.jl:392
[7] +(x::Union{Dates.CompoundPeriod, Dates.Period}) in Dates at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\Dates\src\periods.jl:362
[8] +(A::SparseArrays.AbstractSparseMatrix{Tv, Ti} where {Tv, Ti}, B::LinearAlgebra.Hermitian{var"#s832", var"#s831"} where {var"#s832", var"#s831"<:(SparseArrays.AbstractSparseMatrix{Tv, Ti} where {Tv, Ti})}) in SparseArrays at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\SparseArrays\src\linalg.jl:14
[9] +(A::SparseArrays.AbstractSparseMatrix{Tv, Ti} where {Tv, Ti}, B::LinearAlgebra.Hermitian) in SparseArrays at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\SparseArrays\src\linalg.jl:17
[10] +(A::SparseArrays.AbstractSparseMatrix{Tv, Ti} where {Tv, Ti}, B::LinearAlgebra.Symmetric{var"#s832", var"#s831"} where {var"#s832", var"#s831"<:(SparseArrays.AbstractSparseMatrix{Tv, Ti} where {Tv, Ti})}) in SparseArrays at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\SparseArrays\src\linalg.jl:14
[11] +(A::SparseArrays.AbstractSparseMatrix{Tv, Ti} where {Tv, Ti}, B::LinearAlgebra.Symmetric) in SparseArrays at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\SparseArrays\src\linalg.jl:17
[12] +(A::LinearAlgebra.UpperHessenberg, B::LinearAlgebra.UpperHessenberg) in LinearAlgebra at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\LinearAlgebra\src\hessenberg.jl:101
[13] +(H::LinearAlgebra.UpperHessenberg, J::LinearAlgebra.UniformScaling) in LinearAlgebra at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\LinearAlgebra\src\hessenberg.jl:106
[14] +(J::LinearAlgebra.UniformScaling) in LinearAlgebra at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\LinearAlgebra\src\uniformscaling.jl:150
[15] +(J::LinearAlgebra.UniformScaling, x::Number) in LinearAlgebra at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\LinearAlgebra\src\uniformscaling.jl:145
[16] +(J1::LinearAlgebra.UniformScaling, J2::LinearAlgebra.UniformScaling) in LinearAlgebra at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\LinearAlgebra\src\uniformscaling.jl:151
[17] +(J::LinearAlgebra.UniformScaling, B::BitMatrix) in LinearAlgebra at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\LinearAlgebra\src\uniformscaling.jl:153
[18] +(J::LinearAlgebra.UniformScaling, F::LinearAlgebra.Hessenberg) in LinearAlgebra at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\LinearAlgebra\src\hessenberg.jl:560
[19] +(A::LinearAlgebra.UniformScaling, B::LinearAlgebra.Tridiagonal{var"#s832", V} where {var"#s832"<:Number, V<:AbstractVector{var"#s832"}}) in LinearAlgebra at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\LinearAlgebra\src\special.jl:245
[20] +(A::LinearAlgebra.UniformScaling, B::LinearAlgebra.SymTridiagonal{var"#s832", V} where {var"#s832"<:Number, V<:AbstractVector{var"#s832"}}) in LinearAlgebra at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\LinearAlgebra\src\special.jl:250
[21] +(A::LinearAlgebra.UniformScaling, B::LinearAlgebra.Bidiagonal{var"#s832", V} where {var"#s832"<:Number, V<:AbstractVector{var"#s832"}}) in LinearAlgebra at C:\Users\91863\AppData\Local\Programs\Julia-1.6.1\share\julia\stdlib\v1.6\LinearAlgebra\src\special.jl:25

In [27]:
+(2.0,3)

5.0